In [1]:
import psycopg2
import pandas as pd
import numpy as np
import re
from datetime import datetime
import pytz
import re
from operator import itemgetter

In [2]:
# local postgres connection only
import hidden
sql_string = hidden.psycopg2(hidden.secrets())
print('PostgreSQL connection data taken from hidden.py')

# Make the connection and cursor
conn = psycopg2.connect(sql_string, connect_timeout=3)

PostgreSQL connection data taken from hidden.py


In [3]:
# These are only used for caching
import hashlib
import pickle

def get_candle_data(base_coin, quote_coin):
    h = base_coin+quote_coin
    try:
        with open(f'./cache_data/{h}.pkl', 'rb') as fp:
            print(f"Using cache file: ./cache_data/{h}.pkl")
            return pickle.load(fp)
    except:
        print(f"No cache found")
        pass

    sql = f"""
select *
from
    (select id as the_pair from pairs p where p.coin1='{base_coin}' and p.coin2='{quote_coin}') z
inner join
    candlestick_15m on the_pair=pair_id
where close_time notnull and open_time < '2021-08-07'
order by open_time
"""
    df = pd.read_sql_query(sql, conn)
    with open(f'./cache_data/{h}.pkl', 'wb') as fp:
        print(f"Saving cache to: ./cache_data/{h}.pkl")
        pickle.dump(df, fp, protocol=4)

    return df

In [4]:
dfs = [get_candle_data('ETH', 'BTC'),
       get_candle_data('ETH', 'USDT'),
       get_candle_data('USDT', 'BTC'),
       get_candle_data('USDT', 'ETH'),
       get_candle_data('BTC', 'ETH'),
       get_candle_data('BTC', 'USDT')]
df = pd.concat(dfs)
df = df.set_index(['pair_id','open_time'])

No cache found
Saving cache to: ./cache_data/ETHBTC.pkl
No cache found
Saving cache to: ./cache_data/ETHUSDT.pkl
No cache found
Saving cache to: ./cache_data/USDTBTC.pkl
No cache found
Saving cache to: ./cache_data/USDTETH.pkl
No cache found
Saving cache to: ./cache_data/BTCETH.pkl
No cache found
Saving cache to: ./cache_data/BTCUSDT.pkl


In [5]:
def y_ewm(x):
    x = x.sort_index()
    for i in [8,24,48,96]:
        x[f'close__ewm_{i}'] = (x['close'].ewm(span=i, adjust=False).mean().shift(-i) / x['close'])
    return x

In [6]:
df = df.groupby('pair_id').apply(y_ewm)

In [7]:
y_df = df[[x for x in df.columns if x.startswith('close__')]]

In [8]:
with open(f'./cache_data/targets.pkl', 'wb') as fp:
    pickle.dump(((y_df-1)*20), fp, protocol=4)

In [9]:
df

the_pair          open          high  \
pair_id open_time                                                   
0       2017-07-14 04:00:00         0      0.080000      0.086400   
        2017-07-14 04:15:00         0      0.085289      0.086000   
        2017-07-14 04:30:00         0      0.085811      0.086380   
        2017-07-14 04:45:00         0      0.086314      0.086380   
        2017-07-14 05:00:00         0      0.085874      0.086205   
...                               ...           ...           ...   
1       2021-08-06 22:45:00         1  42643.900000  42785.710000   
        2021-08-06 23:00:00         1  42780.070000  42789.000000   
        2021-08-06 23:15:00         1  42678.950000  42752.530000   
        2021-08-06 23:30:00         1  42719.500000  42841.690000   
        2021-08-06 23:45:00         1  42801.360000  42913.560000   

                                      low         close      volume  \
pair_id open_time                                                     
0       2017-07-14 04:00:00      0.080000      0.086400    8.752000   
        2017-07-14 04:15:00      0.085128      0.085811   61.042000   
        2017-07-14 04:30:00      0.085811      0.086314   53.769000   
        2017-07-14 04:45:00      0.086309      0.086347   42.818000   
        2017-07-14 05:00:00      0.084608      0.084680   16.520000   
...                                   ...           ...         ...   
1       2021-08-06 22:45:00  42571.310000  42780.070000  213.598415   
        2021-08-06 23:00:00  42656.290000  42678.950000  213.887211   
        2021-08-06 23:15:00  42571.430000  42719.500000  223.398210   
        2021-08-06 23:30:00  42699.010000  42801.360000  424.432578   
        2021-08-06 23:45:00  42736.430000  42836.870000  773.687014   

                                         close_time  quote_asset_volume  \
pair_id open_time                                                         
0       2017-07-14 04:00:00 2017-07-14 04:14:59.999        7.282741e-01   
        2017-07-14 04:15:00 2017-07-14 04:29:59.999        5.217079e+00   
        2017-07-14 04:30:00 2017-07-14 04:44:59.999        4.631740e+00   
        2017-07-14 04:45:00 2017-07-14 04:59:59.999        3.697253e+00   
        2017-07-14 05:00:00 2017-07-14 05:14:59.999        1.404228e+00   
...                                             ...                 ...   
1       2021-08-06 22:45:00 2021-08-06 22:59:59.000        9.117914e+06   
        2021-08-06 23:00:00 2021-08-06 23:14:59.000        9.133697e+06   
        2021-08-06 23:15:00 2021-08-06 23:29:59.000        9.531670e+06   
        2021-08-06 23:30:00 2021-08-06 23:44:59.000        1.816148e+07   
        2021-08-06 23:45:00 2021-08-06 23:59:59.000        3.312684e+07   

                             number_of_trades  taker_buy_base_asset_volume  \
pair_id open_time                                                            
0       2017-07-14 04:00:00                26                     3.268000   
        2017-07-14 04:15:00                33                     2.738000   
        2017-07-14 04:30:00                41                    21.225000   
        2017-07-14 04:45:00                61                     7.948000   
        2017-07-14 05:00:00                15                     1.176000   
...                                       ...                          ...   
1       2021-08-06 22:45:00              7798                   112.209132   
        2021-08-06 23:00:00              7844                   106.336888   
        2021-08-06 23:15:00              8041                   118.922631   
        2021-08-06 23:30:00              9422                   225.516483   
        2021-08-06 23:45:00             11349                   432.239849   

                             taker_buy_quote_asset_volume  \
pair_id open_time                                           
0       2017-07-14 04:00:00                  2.823552e-01   
        2017-07-14 04:15:00      